In [83]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from google.colab import files
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_science/train.csv')
test_df=pd.read_csv("/content/drive/MyDrive/data_science/test.csv")

In [ ]:
df.shape

(15730, 16)

In [ ]:
df.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,Flipkart,698,999,30.13%,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,Flipkart,999,1999,50.03%,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,Flipkart,2749,4999,45.01%,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.85%,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,Men Green Sports Sandal,3.9,Men,Flipkart,1379,2299,40.02%,77.0,3.0,35.0,21.0,7.0,7,7,1


In [ ]:
df.describe()

,id,Rating,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
count,15730.000000,15730.000000,15730.000000,15730.000000,15052.000000,15152.000000,15142.000000,15191.000000,15499.000000,15730.000000,15730.000000,15730.000000
mean,10479.541577,4.012873,688.070693,1369.286777,3057.660776,423.976307,1585.239466,655.923310,357.260662,155.085188,275.500572,0.601526
std,6080.166276,0.298440,649.409586,1240.900227,11846.965689,1768.230384,6177.476241,2855.735531,1402.246610,558.650254,958.589075,0.489600
min,3.000000,0.000000,69.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5212.000000,3.900000,349.000000,699.000000,63.000000,9.000000,30.000000,12.000000,7.000000,3.000000,6.000000,0.000000
50%,10458.500000,4.000000,474.000000,999.000000,308.000000,44.000000,150.000000,60.000000,34.000000,17.000000,30.000000,1.000000
75%,15766.750000,4.200000,699.000000,1299.000000,1526.000000,215.000000,788.000000,300.000000,172.000000,77.000000,140.000000,1.000000
max,20973.000000,5.000000,5998.000000,13499.000000,289973.000000,45448.000000,151193.000000,74037.000000,34978.000000,11705.000000,18060.000000,1.000000


In [ ]:
df.dtypes

id              int64
title          object
Rating        float64
maincateg      object
platform       object
price1          int64
actprice1       int64
Offer %        object
norating1     float64
noreviews1    float64
star_5f       float64
star_4f       float64
star_3f       float64
star_2f         int64
star_1f         int64
fulfilled1      int64
dtype: object

In [ ]:
df['Offer %']=df['Offer %'].str.slice(0,-1).astype(float)

In [ ]:
df.dtypes

id              int64
title          object
Rating        float64
maincateg      object
platform       object
price1          int64
actprice1       int64
Offer %       float64
norating1     float64
noreviews1    float64
star_5f       float64
star_4f       float64
star_3f       float64
star_2f         int64
star_1f         int64
fulfilled1      int64
dtype: object

In [ ]:
df["New"]=df['actprice1']-df['actprice1']*df['Offer %']/100

In [ ]:
df['New']=df['New']-df['price1']
df.New

0         0.0013
1        -0.0997
2        -0.0499
3        91.2460
4        -0.0598
          ...   
15725     0.0071
15726     0.0000
15727    -0.0093
15728     0.0501
15729    -0.1098
Name: New, Length: 15730, dtype: float64

In [ ]:
df.New.mean()

0.07173392879847636

In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/data_science/test.csv')

In [ ]:
df_test.head()

,id,title,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,2242,Casuals For Men (Blue),3.8,Men,Flipkart,999,27928,3543,14238.0,4295,3457,1962,3976.0,1
1,20532,Women Black Flats Sandal,3.9,Women,Flipkart,499,3015,404,1458.0,657,397,182,321.0,1
2,10648,Women Gold Wedges Sandal,3.9,Women,Flipkart,999,449,52,229.0,70,71,33,46.0,1
3,20677,Men's Height Increasing High Heel Formal Party...,3.9,Men,Flipkart,2999,290,40,141.0,51,49,17,32.0,1
4,12593,Loafers For Men (Tan),3.9,Men,Flipkart,999,2423,326,1265.0,414,293,143,308.0,0


In [ ]:
offer_percentage = df['Offer %']

In [ ]:

X =df.drop(['New','Offer %','id','title'], axis=1)
X2=df.drop(['New','Offer %','id'], axis=1)
X_test = df_test.drop(['id'], axis=1)

In [ ]:
X2.shape

(15730, 14)

In [ ]:
X_test.isnull().sum()

title           0
Rating        203
maincateg      67
platform        0
actprice1       0
norating1       0
noreviews1      0
star_5f        68
star_4f         0
star_3f         0
star_2f         0
star_1f       186
fulfilled1      0
dtype: int64

In [ ]:
X_test['Rating'] = X_test['Rating'].fillna(X_test['Rating'].median())
X_test['star_5f'] = X_test['star_5f'].fillna(X_test['star_5f'].median())
X_test['star_1f'] = X_test['star_1f'].fillna(X_test['star_1f'].median())

In [ ]:
X

,Rating,maincateg,platform,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,3.9,Women,Flipkart,698,999,38.0,7.0,17.0,9.0,6.0,3,3,0
1,3.8,Men,Flipkart,999,1999,531.0,69.0,264.0,92.0,73.0,29,73,1
2,4.4,Women,Flipkart,2749,4999,17.0,4.0,11.0,3.0,2.0,1,0,1
3,4.2,Men,Flipkart,518,724,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,3.9,Men,Flipkart,1379,2299,77.0,3.0,35.0,21.0,7.0,7,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,4.3,Women,Flipkart,567,1199,807.0,114.0,485.0,177.0,61.0,41,43,0
15726,3.9,Women,Flipkart,499,998,246.0,34.0,120.0,45.0,37.0,16,28,1
15727,3.7,Women,Flipkart,329,499,NaN,18.0,NaN,NaN,20.0,10,15,1
15728,3.9,Men,Amazon,2249,4499,750.0,479.0,13.0,6.0,10.0,25,47,1


In [ ]:
X.platform.value_counts()

Flipkart    15389
Amazon        341
Name: platform, dtype: int64

In [ ]:
X.maincateg.value_counts()

Women    8781
Men      6423
Name: maincateg, dtype: int64

In [ ]:
df.isnull().sum()

id              0
title           0
Rating          0
maincateg     526
platform        0
price1          0
actprice1       0
Offer %         0
norating1     678
noreviews1    578
star_5f       588
star_4f       539
star_3f       231
star_2f         0
star_1f         0
fulfilled1      0
New             0
dtype: int64

In [ ]:
X2.isnull().sum()

title           0
Rating          0
maincateg     526
platform        0
price1          0
actprice1       0
norating1     678
noreviews1    578
star_5f       588
star_4f       539
star_3f       231
star_2f         0
star_1f         0
fulfilled1      0
dtype: int64

In [ ]:
X2['Rating'] = X2['Rating'].fillna(X2['Rating'].median())
X2['star_5f'] = X2['star_5f'].fillna(X2['star_5f'].median())
X2['star_1f'] = X2['star_1f'].fillna(X2['star_1f'].median())

In [ ]:
X2.isnull().sum()

title           0
Rating          0
maincateg     526
platform        0
price1          0
actprice1       0
norating1     678
noreviews1    578
star_5f         0
star_4f       539
star_3f       231
star_2f         0
star_1f         0
fulfilled1      0
dtype: int64

In [ ]:
X2['norating1'] = X2['norating1'].fillna(X2['norating1'].median())
X2['noreviews1'] = X2['noreviews1'].fillna(X2['noreviews1'].median())
X2['star_3f'] = X2['star_3f'].fillna(X2['star_3f'].median())
X2['star_4f'] = X2['star_4f'].fillna(X2['star_4f'].median())

In [ ]:
X2.isnull().sum()

title           0
Rating          0
maincateg     526
platform        0
price1          0
actprice1       0
norating1       0
noreviews1      0
star_5f         0
star_4f         0
star_3f         0
star_2f         0
star_1f         0
fulfilled1      0
dtype: int64

In [ ]:
temp = X2[X2.maincateg.isnull()]

In [ ]:
X2[X2.maincateg.isnull()]

,title,Rating,maincateg,platform,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
19,Women Beige Heels Sandal,4.0,NaN,Flipkart,499,999,308.0,44.0,28.0,9.0,10.0,2,5,0
38,"Men Brown, Orange Sports Sandal",4.1,NaN,Flipkart,819,999,26640.0,3667.0,14148.0,60.0,34.0,1113,2094,1
68,Women Copper Flats Sandal,4.1,NaN,Flipkart,349,999,308.0,27.0,150.0,60.0,22.0,5,15,1
91,Slippers,3.6,NaN,Flipkart,213,249,308.0,44.0,150.0,60.0,466.0,281,476,1
116,Ace Slip-On Running Shoes For Men (Black),4.3,NaN,Flipkart,2999,4999,308.0,44.0,46.0,27.0,5.0,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15536,Slides,4.5,NaN,Flipkart,499,799,308.0,1.0,23.0,60.0,1.0,1,2,0
15570,Women Multicolor Bellies Sandal,3.7,NaN,Flipkart,501,999,308.0,44.0,150.0,141.0,111.0,75,128,1
15583,Jutis For Men (Black),3.7,NaN,Flipkart,494,499,308.0,44.0,22.0,60.0,4.0,4,8,0
15674,Zod Runner V3 Wn s IDP Running Shoes For Women...,4.3,NaN,Flipkart,1699,3999,308.0,44.0,150.0,60.0,268.0,102,140,1


In [ ]:
temp2=X2[X2.title.str.find('Men')>=0]
temp3=X2[X2.title.str.find('Women')>=0]      

In [ ]:
for row in temp2.index:
  X2.at[row, 'maincateg'] = 'Men'

In [ ]:
for row in temp3.index:
  X2.at[row, 'maincateg'] = 'Women'

In [ ]:
X2.isnull().sum()

title          0
Rating         0
maincateg     61
platform       0
price1         0
actprice1      0
norating1      0
noreviews1     0
star_5f        0
star_4f        0
star_3f        0
star_2f        0
star_1f        0
fulfilled1     0
dtype: int64

In [ ]:
X2['maincateg'] = X2['maincateg'].fillna('Women')

In [ ]:
X2.isnull().sum()

title         0
Rating        0
maincateg     0
platform      0
price1        0
actprice1     0
norating1     0
noreviews1    0
star_5f       0
star_4f       0
star_3f       0
star_2f       0
star_1f       0
fulfilled1    0
dtype: int64

In [ ]:
X2=X2.drop('title',axis=1)

In [ ]:
X2.isnull().sum()

Rating        0
maincateg     0
platform      0
price1        0
actprice1     0
norating1     0
noreviews1    0
star_5f       0
star_4f       0
star_3f       0
star_2f       0
star_1f       0
fulfilled1    0
dtype: int64

In [ ]:
X_test.isnull().sum()

title          0
Rating         0
maincateg     67
platform       0
actprice1      0
norating1      0
noreviews1     0
star_5f        0
star_4f        0
star_3f        0
star_2f        0
star_1f        0
fulfilled1     0
dtype: int64

In [ ]:
temp4=X_test[X_test.title.str.find('Men')>=0]
temp5=X_test[X_test.title.str.find('Women')>=0]     

In [ ]:
for row in temp4.index:
  X_test.at[row, 'maincateg'] = 'Men'

In [ ]:
for row in temp5.index:
  X_test.at[row, 'maincateg'] = 'Women'

In [ ]:
X_test.isnull().sum()

title         0
Rating        0
maincateg     7
platform      0
actprice1     0
norating1     0
noreviews1    0
star_5f       0
star_4f       0
star_3f       0
star_2f       0
star_1f       0
fulfilled1    0
dtype: int64

In [ ]:
X_test['maincateg'] = X_test['maincateg'].fillna('Women')

In [ ]:
X_test.isnull().sum()

title         0
Rating        0
maincateg     0
platform      0
actprice1     0
norating1     0
noreviews1    0
star_5f       0
star_4f       0
star_3f       0
star_2f       0
star_1f       0
fulfilled1    0
dtype: int64

In [ ]:
X2.shape

(15730, 13)

In [ ]:
X_test.shape

(5244, 13)

In [ ]:
X_test = X_test.drop('title',axis=1)

In [ ]:
X_test

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,3.8,Men,Flipkart,999,27928,3543,14238.0,4295,3457,1962,3976.0,1
1,3.9,Women,Flipkart,499,3015,404,1458.0,657,397,182,321.0,1
2,3.9,Women,Flipkart,999,449,52,229.0,70,71,33,46.0,1
3,3.9,Men,Flipkart,2999,290,40,141.0,51,49,17,32.0,1
4,3.9,Men,Flipkart,999,2423,326,1265.0,414,293,143,308.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5239,4.0,Women,Flipkart,699,1235,153,156.0,323,155,62,29.0,1
5240,3.9,Men,Flipkart,1993,329,56,350.0,37,60,5,37.0,0
5241,3.8,Women,Flipkart,999,1280,135,574.0,290,172,94,150.0,0
5242,3.9,Women,Flipkart,499,4867,574,2384.0,974,648,328,533.0,0


In [ ]:
df.maincateg = df.maincateg.str.lower()
df.title = df.title.str.lower()

In [ ]:
df.shape

(15730, 17)

In [ ]:
df.dropna(how='any').shape

(14945, 17)

In [ ]:
X=X.dropna(how='any')

In [ ]:
X.shape

(14945, 13)

In [ ]:
X.head()

,Rating,maincateg,platform,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,3.9,Women,Flipkart,698,999,38.0,7.0,17.0,9.0,6.0,3,3,0
1,3.8,Men,Flipkart,999,1999,531.0,69.0,264.0,92.0,73.0,29,73,1
2,4.4,Women,Flipkart,2749,4999,17.0,4.0,11.0,3.0,2.0,1,0,1
3,4.2,Men,Flipkart,518,724,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,3.9,Men,Flipkart,1379,2299,77.0,3.0,35.0,21.0,7.0,7,7,1


In [ ]:
temp = pd.get_dummies(data=X)
print(temp.shape)
temp.head()
X=temp

(14945, 15)


In [ ]:
temp = pd.get_dummies(data=X2)
print(temp.shape)
temp.head()
X2=temp

(15730, 15)


In [ ]:
temp = pd.get_dummies(data=X_test)
print(temp.shape)
temp.head()
X_test=temp

(5244, 14)


In [86]:
Y2=df.price1

In [ ]:
Y2.shape
X3=X2.drop(['maincateg_Men','platform_Amazon'],axis=1)
X3_test=X_test.drop(['maincateg_Men','platform_Amazon'],axis=1)

In [ ]:
X3

,Rating,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,maincateg_Women,platform_Flipkart
0,3.9,999,38.0,7.0,17.0,9.0,6.0,3,3,0,1,1
1,3.8,1999,531.0,69.0,264.0,92.0,73.0,29,73,1,0,1
2,4.4,4999,17.0,4.0,11.0,3.0,2.0,1,0,1,1,1
3,4.2,724,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1,0,1
4,3.9,2299,77.0,3.0,35.0,21.0,7.0,7,7,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
15725,4.3,1199,807.0,114.0,485.0,177.0,61.0,41,43,0,1,1
15726,3.9,998,246.0,34.0,120.0,45.0,37.0,16,28,1,1,1
15727,3.7,499,308.0,18.0,150.0,60.0,20.0,10,15,1,1,1
15728,3.9,4499,750.0,479.0,13.0,6.0,10.0,25,47,1,0,0


In [ ]:
X3_test

,Rating,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,maincateg_Women,platform_Flipkart
0,3.8,999,27928,3543,14238.0,4295,3457,1962,3976.0,1,0,1
1,3.9,499,3015,404,1458.0,657,397,182,321.0,1,1,1
2,3.9,999,449,52,229.0,70,71,33,46.0,1,1,1
3,3.9,2999,290,40,141.0,51,49,17,32.0,1,0,1
4,3.9,999,2423,326,1265.0,414,293,143,308.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5239,4.0,699,1235,153,156.0,323,155,62,29.0,1,1,1
5240,3.9,1993,329,56,350.0,37,60,5,37.0,0,0,1
5241,3.8,999,1280,135,574.0,290,172,94,150.0,0,1,1
5242,3.9,499,4867,574,2384.0,974,648,328,533.0,0,1,1


In [ ]:
xgb1 = XGBRegressor()
parameters = {'learning_rate': [0.01, 0.05, 0.1], #so called `eta` value
              'max_depth': [3,5, 7,9,10,13,15],
              'base_score':[0.25,0.50] ,
              'colsample_bytree': [0.3,0.7],
              'n_estimators': [100,500,900]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True,
                        scoring=["r2","neg_root_mean_squared_error"],
                        refit="r2")

xgb_grid.fit(X3,Y2)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits
[04:12:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, estimator=XGBRegressor(), n_jobs=5,
             param_grid={'base_score': [0.25, 0.5],
                         'colsample_bytree': [0.3, 0.7],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7, 9, 10, 13, 15],
                         'n_estimators': [100, 500, 900]},
             refit='r2', scoring=['r2', 'neg_root_mean_squared_error'],
             verbose=True)

In [ ]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

0.9115851258909522
{'base_score': 0.25, 'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 9, 'n_estimators': 900}


In [81]:
xgbr=XGBRegressor(base_score=0.25, max_depth=10, learning_rate=0.05,colsample_bytree=0.7,
             n_estimators=900)

In [82]:

xgbr.fit(X3,Y2);
xgbr_pred=xgbr.predict(X3_test);
price1 = pd.Series(xgbr_pred)
frame = { 'id': test_df.id, 'price1': price1 }
submissionxgbr = pd.DataFrame(frame)
submissionxgbr.to_csv('submissionxgbr.csv', index=False)
files.download('submissionxgbr.csv')

[04:45:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>